In [1]:
import pickle,sys,os,lda,scipy,pandas
import numpy as np
from nltk import word_tokenize, sent_tokenize
bios = []
for suf in ['18000']:
    bios += pandas.read_pickle("train-corpus/corpus"+suf+".pickle")

//anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
def progress(i, end_val, bar_length=50):
    percent = float(i) / end_val
    hashes = '#' * int(round(percent * bar_length))
    spaces = ' ' * (bar_length - len(hashes))
    sys.stdout.write("\r{0} / {1} Percent: [{2}] {3}%".format(i, end_val, hashes + spaces, int(round(percent * 100))))
    sys.stdout.flush()

In [93]:
bios[0]

[('Samuel Cornish',
  [('Summary',
    "Samuel Eli Cornish (1795  6 November 1858) was an American Presbyterian minister, abolitionist, publisher, and journalist.  He was a leader in New York City's small free black community, where he organized the first congregation of black Presbyterians in New York. In 1827 he became one of two editors of the newly founded ''Freedom's Journal'', the first black newspaper in the United States. In 1833 he was a founding member of the interracial American Anti-Slavery Society.\n\n"),
   ('Career',
    "\nWhen Cornish was ordained in 1822, his parish was officially established as the New Demeter Street Presbyterian Church, making it the first black Presbyterian Church in New York City. He later ministered at the First African Presbyterian Church in Philadelphia, and Emmanuel Church in New York City. Cornish held high-ranking positions within the American Bible Society and the American Missionary Association, founded in 1846. He was one of the four foun

## Find noise words (in order to filter them later)

In [4]:
tokens_freqs = dict()

i = 0
for bio in bios:
    for segment in bio[0][1]:
        
        for paragraph_text in segment[1].split('\n'):
            tokens = word_tokenize(paragraph_text)
            for token in tokens:
                if token in tokens_freqs:
                    tokens_freqs[token] += 1
                else:
                    tokens_freqs[token] = 1
    i += 1
    progress(i, len(bios))

1000 / 1000 Percent: [##################################################] 100%

In [5]:
import operator

# Sorder tokens by frequency in reverse order (most frequent first):
most_frequent_tokens = sorted(tokens_freqs.items(), key=operator.itemgetter(1))
most_frequent_tokens.reverse()

noise_words = set()
for token_and_freq in most_frequent_tokens:
    # We take the threshold to be 1/100 the frequency of the most frequnent token:
    if token_and_freq[1] > most_frequent_tokens[0][1] / 100:
        noise_words.add(token_and_freq[0])
    else:
        break

## Collect paragraph data for every biography

In [6]:
bios_df = list()
all_tokens = set()
i = 0
for bio in bios:
    data = {
        'person' : bio[0][0],
        'tokenized_paragraphs' : list(),
        'paragraph_splits' : list(),
        'word_splits': [0],
        'length' : 0,
        'segments' : 0
    }
    number_of_words = 0
    for segment in bio[0][1]:
        number_of_paragraphs = 0
        data['segments'] += 1
        for paragraph_text in segment[1].split('\n'):
            original_tokens = word_tokenize(paragraph_text)
            tokens = list()
            for token in original_tokens:
                if token not in noise_words:
                    tokens.append(token)
            
            if len(tokens) > 0:
                number_of_paragraphs += 1
                number_of_words += len(tokens)
                all_tokens |= set(tokens)
                data['tokenized_paragraphs'].append(tokens)  
        data['paragraph_splits'].append(number_of_paragraphs)
        data['word_splits'].append(number_of_words)
    data['length'] = number_of_words
    bios_df.append(data)
    i += 1
    progress(i, len(bios))

1000 / 1000 Percent: [##################################################] 100%

In [7]:
bios_data = pandas.DataFrame(bios_df)
bios_data[:2]

,length,paragraph_splits,person,segments,tokenized_paragraphs,word_splits
0,176,"[1, 4]",Samuel Cornish,2,"[[Samuel, Eli, Cornish, 1795, 6, 1858, Presbyt...","[0, 34, 176]"
1,73,"[1, 2]",John Renshaw Starr,2,"[[Renshaw, Starr, 1996, sons, Alfred, Demarest...","[0, 25, 73]"


In [8]:
all_tokens_list = list(all_tokens)
number_of_tokens = len(all_tokens_list)
all_paragraphs = bios_data['tokenized_paragraphs'].sum()
paragraphs_bow = np.zeros([len(all_paragraphs),number_of_tokens], dtype = np.int)
tokens_indices_dict = dict()
for i in range(number_of_tokens):
    tokens_indices_dict[all_tokens_list[i]] = i
    
for i in range(len(all_paragraphs)):
    for w in all_paragraphs[i]:
        paragraphs_bow[i][tokens_indices_dict[w]] += 1
        
    progress(i + 1, len(all_paragraphs))

10272 / 10272 Percent: [##################################################] 100%

## Assign LDA topics to paragraphs with word embeddings

### Generate topic lists

### Using gensim lda:

In [9]:
from gensim import corpora

dictionary = corpora.Dictionary(all_paragraphs)
corpus = [dictionary.doc2bow(text) for text in all_paragraphs]

In [10]:
from gensim.models import LdaModel

lda = LdaModel(corpus, id2word=dictionary, num_topics=20, passes=20)

In [17]:
topics_words = list()
number_of_topics = 20

i = 0
for topic in lda.show_topics(num_topics=number_of_topics, formatted=False, num_words=len(all_tokens)):
    i += 1
    topic_words = list()
    for score, word in topic:
        topic_words.append(word)
    print('Topic {}: {}'.format(i, ' '.join(topic_words[:15])))
    topics_words.append(topic_words)

Topic 1: life political both said there than some so did This out them any world could
Topic 2: 2009 DJ File computer Sciences Rudd former computing Angels Minister Most Prime George ''Adventure International
Topic 3: Mercier Quebec named *The engine Canada According memory Dayton power government effort being case Chinese
Topic 4: worked Berlin several game band moved Company California original Frank Island home label recorded performed
Topic 5: California Art elected U.S. Senate Party losing active election former House Washington Republican both leader
Topic 6: Princeton 36 67 Duke Society der artists Award No Poland player Royal ** Symphony Reformed
Topic 7: Robert Life wife King title father instruments starred author 1956 Mihailovic By musical William George
Topic 8: Open Award 2014 46 2012 2010 2011 2008 64 Australian Best won 2006 26 ISBN
Topic 9: Music CD television played role career playing appeared Hurley 1995 directed marriage Love novel character
Topic 10: Lloyd Hall B W

In [12]:
topic_words_indices_dicts = list()
for topic_words in topics_words:
    topic_words_indices_dict = dict()
    for i in range(len(topic_words)):
        topic_words_indices_dict[topic_words[i]] = i
    topic_words_indices_dicts.append(topic_words_indices_dict)

In [48]:
# topic_words_indices_dicts[2]['Samuel']

In [49]:
def calculate_word_topic_id(word, topics_words):
    topic_id = -1
    min_topic_index = number_of_tokens + 1
    for current_topic_id in range(number_of_topics):
        index = topic_words_indices_dicts[current_topic_id][word]
        if index < min_topic_index:
            topic_id = current_topic_id
            min_topic_index = index
            
    return topic_id

In [50]:
def calculate_vocabulary_topic_ids(vocabulary, topics_words):
    topic_ids = list()
    for word in vocab:
        topic_ids.append(calculate_word_topic_id(word, topics_words))
        
    return topic_ids

In [51]:
vocab = all_tokens_list

In [52]:
vocabulary_topic_ids = calculate_vocabulary_topic_ids(vocab, topics_words)

In [73]:
def get_vocabulary_word_topic_id(vocabulary, vocabulary_topic_ids, word):
    if word in vocabulary:
        return vocabulary_topic_ids[vocabulary.index(word)]
    return None

In [74]:
get_vocabulary_word_topic_id(vocab, vocabulary_topic_ids, "Samuel")

6

In [75]:
def get_sentence_topic_ids(sentence, topics_words):
    topic_ids = list()
    for word in sentence:
        topic_id = get_vocabulary_word_topic_id(vocab, vocabulary_topic_ids, word)
        if topic_id is not None:
            topic_ids.append(topic_id)
        
    return topic_ids

In [76]:
def get_sentences_topic_ids(sentences, topic_words):
    topic_ids = list()
    for sentence in sentences:
        topic_ids.append(get_sentence_topic_ids(sentence, topics_words))
        
    return topic_ids

In [77]:
def get_biography_sentences_topic_ids(biography_sentences_tokens, topic_words):
    biography_sentences_topic_ids = list()
    for sentences in biography_sentences_tokens:
        biography_sentences_topic_ids.append(get_sentences_topic_ids(sentences, topic_words))
    
    return biography_sentences_topic_ids

In [79]:
def calculate_sentence_topics_frequency(sentence_topic_ids, number_of_topics):
    topics_frequencies = np.zeros(number_of_topics)
    for topic_id in sentence_topic_ids:
        topics_frequencies[topic_id] += 1
    
    return topics_frequencies

In [80]:
def calculate_sentences_topics_frequency(sentences_topic_ids, number_of_topics):
    sentences_topics_frequencies = list()
    for sentence in sentences_topic_ids:
        sentences_topics_frequencies.append(calculate_sentence_topics_frequency(sentence, number_of_topics))
        
    return sentences_topics_frequencies

In [81]:
def calculate_biography_sentences_topics_frequencies(biography_sentences_topic_ids, number_of_topics):
    biography_sentences_topics_frequencies = list()
    for sentences in biography_sentences_topic_ids:
        biography_sentences_topics_frequencies.append(calculate_sentences_topics_frequency(sentences, number_of_topics))
        
    return biography_sentences_topics_frequencies

In [83]:
def calculate_coherence_score(sentences_topics_frequencies, number_of_topics, position, window_size):
    pre_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        pre_topics_frequencies += sentences_topics_frequencies[position - i]
        
    post_topics_frequencies = np.zeros(number_of_topics)
    for i in range(window_size):
        post_topics_frequencies += sentences_topics_frequencies[position + 1 + i]
        
    dot_product = np.dot(pre_topics_frequencies, post_topics_frequencies)
    pre_norm = np.linalg.norm(pre_topics_frequencies)
    post_norm = np.linalg.norm(post_topics_frequencies)
    
    cosine_similarity = dot_product / (pre_norm * post_norm)
    
    return cosine_similarity

In [85]:
def calculate_coherence_scores(sentences_topics_frequencies, number_of_topics, window_size):
    coherence_scores = list()
    for i in range(len(sentences_topics_frequencies) - 2 * window_size):
        score = calculate_coherence_score(sentences_topics_frequencies, number_of_topics, window_size + i - 1, window_size)
        coherence_scores.append(score)
        
    return coherence_scores

In [86]:
def calculate_depth_scores(coherence_scores):
    depth_scores = list()
    for i in range(len(coherence_scores)):
        hl = coherence_scores[i]
        hr = coherence_scores[i]
        for j in range(i):
            if coherence_scores[i - j] > coherence_scores[i]:
                hl = coherence_scores[i - j]
            else:
                break
        
        for j in range(i + 1, len(coherence_scores) - i):
            if coherence_scores[j] > coherence_scores[i]:
                hr = coherence_scores[j]
            else:
                break
                
        depth_score = 0.5 * (hl + hr - 2 * coherence_scores[i])
        depth_scores.append(depth_score)
        
    return depth_scores

In [102]:
biography_sentences_topic_ids = get_biography_sentences_topic_ids(bios_df[1]['tokenized_paragraphs'], topic_words)
biography_sentences_topics_frequencies = calculate_biography_sentences_topics_frequencies(biography_sentences_topic_ids, number_of_topics)

In [112]:
bios_df[1]['tokenized_paragraphs']

[['Renshaw',
  'Starr',
  '1996',
  'sons',
  'Alfred',
  'Demarest',
  'Starr',
  'Ethel',
  'Renshaw',
  'English',
  'grandson',
  'William',
  'Robert',
  'Renshaw',
  'artist',
  'soldier',
  'Second',
  'story',
  'told',
  'book',
  'Starr',
  'Affair',
  'Jean',
  'Overton',
  'Fuller'],
 ['By',
  'exploiting',
  'ability',
  'pass',
  'himself',
  'off',
  'Frenchman',
  'joined',
  'group',
  'French',
  'Belgian',
  'prisoners',
  'released',
  'custody',
  'Red',
  'Cross',
  'taken',
  'Switzerland',
  'war',
  'Europe',
  'drew',
  'close'],
 ['Stories',
  'SOE',
  'agents',
  'shared',
  'captivity',
  'Avenue',
  'Foch',
  'resulted',
  'doubts',
  'being',
  'raised',
  'loyalty',
  'case',
  'subject',
  'MI5',
  'investigation',
  'concluded',
  'although',
  'behaviour',
  'certainly',
  'suspicious',
  'there',
  'no',
  'grounds',
  'criminal',
  'prosecution']]

In [105]:
coherence_scores = calculate_coherence_scores(biography_sentences_topics_frequencies[0], number_of_topics, 3)

In [106]:
depth_scores = calculate_depth_scores(coherence_scores)

In [107]:
mean = np.mean(depth_scores)

In [108]:
std = np.std(depth_scores)

In [109]:
mean - std /2

-0.0029486130296959706

In [110]:
depth_scores

[0.0,
 0.0,
 0.11815613696359856,
 0.010876058127594956,
 0.0,
 0.1228966707500575,
 0.0017059410337709746,
 0.0,
 0.045401051710649343,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [101]:
bios[1]

[('John Renshaw Starr',
  [('Summary',
    "John Renshaw Starr (died 1996), was one of two sons of Alfred Demarest Starr (an American) and Ethel Renshaw (English). He was a grandson of William Robert Renshaw. He was an artist and a soldier during the Second World War. His story is told in a book, ''The Starr Affair'', by Jean Overton Fuller.\n\n"),
   ('Release',
    'By exploiting his ability to pass himself off as a Frenchman, he joined a group of French and Belgian prisoners who were released into the custody of the Red Cross and taken to Switzerland as the war in Europe drew to a close.\n\nStories from other SOE agents who shared his captivity at the Avenue Foch resulted in doubts being raised about his loyalty, and his case became the subject of an MI5 investigation, which concluded that although his behaviour was certainly suspicious, there were no grounds for criminal prosecution. \n\n')])]

In [114]:
len(biography_sentences_topics_frequencies)

3